本文件用于对外部数据进行预处理，得到风格因子的收益和行业收益数据

In [ ]:
from sklearn.linear_model import LinearRegression
import os
import pandas as pd
from mysystem.backtest import internal_data_loader

pctdf,data = internal_data_loader.load_processed_prices()

由于外部数据过大，超过作业要求，因此没有附在文件中，只将处理好的结果放在newdata文件夹

In [ ]:
d = pd.read_feather('./daily_risk_byday/20200103.ftr')
def f(x):
    if x[-1] == 'G':
        return x[:-4] + 'SH'
    else:
        return x[:-4] + 'SZ'
d['order_book_id'] = d['order_book_id'].apply(f)
names = d.columns[2:]
names

In [ ]:
output = pd.DataFrame(columns=names,index=os.listdir('./daily_risk_byday/'))

for i in os.listdir('./daily_risk_byday/'):
    
    try:
        d = pd.read_feather('./daily_risk_byday/'+i)
        def f(x):
            if x[-1] == 'G':
                return x[:-4] + 'SH'
            else:
                return x[:-4] + 'SZ'
        d['order_book_id'] = d['order_book_id'].apply(f)
        cur_pct = pctdf.loc[d['datetime'].astype('datetime64[ns]')[0],:]
        
        for j in names:
            cur = d.set_index('order_book_id')
            cur_ret = (cur[j] * cur_pct).dropna().sum() 
            
            # 将两个 Series 合并为一个 DataFrame
            data = pd.concat([cur[j], cur_pct], axis=1)
            data.columns = ['X', 'Y']

            # 删除含有缺失值的行
            data.dropna(subset=['X', 'Y'], inplace=True)
            
            # 获取 X 和 Y 列作为训练数据
            X = data['X'].values.reshape(-1, 1)
            y = data['Y'].values

            # 初始化线性回归模型
            model = LinearRegression()

            # 拟合模型
            model.fit(X, y)

            output.loc[i,j] = model.coef_[0]
    except:
        pass

最终结果存储于daily_risk.csv

In [ ]:
output.index = pd.Series(output.index).apply(lambda x:pd.to_datetime(x[:-4]))
del output['comovement']

output.dropna().to_csv('daily_risk.csv')